In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import copy
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go

In [11]:
import plotly
print(plotly.__version__)

5.5.0


In [2]:
PATH_MAIN ="/Users/ilariabonavitacorp/Library/CloudStorage/OneDrive-TrilateralResearchLtd/trilateralResearch/TechEthos/media_analysis/analysis/"
PATH_KW = "/Users/ilariabonavitacorp/Library/CloudStorage/OneDrive-TrilateralResearchLtd/trilateralResearch/TechEthos/media_analysis/resources/list_of_keywords_final.xlsx"



In [3]:
countries = ['at','cz','de','es','fr','gb','ie','it','nl','ro','rs','se','us']

In [4]:
countries_dict ={ 'at':{'lang':'GERMAN_AT','lang_code':'de','name':'Austria'},
                  'cz':{'lang':'CZECH','lang_code':'cs','name':'Czech Republic'},
                  'de':{'lang':'GERMAN_DE','lang_code':'de','name':'Germany'},
                  'es':{'lang':'SPANISH','lang_code':'es','name':'Spain'},
                  'fr':{'lang':'FRENCH','lang_code':'fr','name':'France'},
                  'gb':{'lang':'ENGLISH','lang_code':'en','name':'United Kindgdom'},
                  'ie':{'lang':'ENGLISH','lang_code':'en','name':'Ireland'},
                  'it':{'lang':'ITALIAN','lang_code':'it','name':'Italy'},
                  'nl':{'lang':'DUTCH','lang_code':'nl','name':'The Netherlands'},
                  'ro':{'lang':'ROMANIAN','lang_code':'ro','name':'Romania'},
                  'rs':{'lang':'SERBIAN','lang_code':'rs','name':'Serbia'},
                  'se':{'lang':'SWEDISH','lang_code':'se','name':'Sweden'},
                  'us':{'lang':'ENGLISH','lang_code':'en','name':'United States'},
                }

In [5]:
topics = ['climate', 'digitalxr','neurotech']

In [6]:
# load the keywords per topic and country in a dict
main_kw_dict = dict.fromkeys(countries_dict.keys())
for c in countries_dict.keys():
    language = countries_dict[c]['lang']
    kw_dict = dict.fromkeys(topics + ['ethics'])
    for t in topics + ['ethics']:
        kw_dict[t] = pd.read_excel(PATH_KW,sheet_name=t)
    
    topic_kw = {"climate": get_keywords_country_topic_list(kw_dict, 'climate', language),
            
                "digitalxr" : get_keywords_country_topic_list(kw_dict, 'digitalxr', language),
            
                "neurotech" : get_keywords_country_topic_list(kw_dict, 'neurotech', language),
           
            "ethics" :  get_keywords_country_topic_list(kw_dict, 'ethics', language) }
    
    main_kw_dict[c] = topic_kw
        

NameError: name 'get_keywords_country_topic_list' is not defined

In [8]:
dfs = []
for c in countries:
    PATH_FILE = PATH_MAIN + c + '/data_processed/df_all_final.xlsx'
    df_tmp = pd.read_excel(PATH_FILE,index_col=0)
    if c== 'ro':
        df_tmp['kw_matches'] = df_tmp['kw_matches_merged']
    print(c)
    print(df_tmp.shape)
    dfs.append(df_tmp)

at
(173, 37)
cz
(203, 37)
de
(1195, 39)
es
(489, 39)
fr
(2091, 39)
gb
(1651, 45)
ie
(940, 39)
it
(1879, 45)
nl
(246, 45)
ro
(617, 48)
rs
(311, 37)
se
(334, 37)
us
(3423, 39)


In [9]:
df_all = pd.concat(dfs)

In [10]:
df_all.shape

(13552, 56)

In [11]:
df_all = df_all[df_all.columns[df_all.columns.isin(['topic', 'title', 'url', 'date', 'text', 'country', 'year', 'month',
       'day', 'domain', 'section', 'kw_matches', 'kw_matches_count',
                                                    'kw_unique', 'kw_unique_count','ethics_matches'])]]


In [12]:
df_all = df_all.reset_index(drop=True)

In [13]:
len(df_all[(df_all['date'].dt.month==1) & (df_all['date'].dt.year==2020)])

14

In [302]:
df_topics_ym_count = df_all.groupby([df_all["date"].dt.to_period('M'),"topic"]).size().unstack('topic', fill_value=0).reset_index()
df_topics_ym_count_melt = df_topics_ym_count.melt(id_vars = 'date',value_vars =['climate','digitalxr','neurotech']
                                                 )

df_topics_ym_count_melt['datetransf'] = pd.to_datetime(df_topics_ym_count_melt.date.astype(str), format='%Y-%M')
dtnp = list(df_topics_ym_count.date.values.astype('datetime64[M]'))
dates = list(pd.to_datetime(df_topics_ym_count.date.astype(str), format='%Y-%M'))
fig = go.Figure(data=[
    go.Bar(name='climate', x= dtnp, y=list(df_topics_ym_count.climate), marker={'color': px.colors.qualitative.Prism[4]}),
    go.Bar(name='digitalxr', x= dtnp, y=list(df_topics_ym_count.digitalxr), marker={'color': px.colors.qualitative.Prism[1]}),
    go.Bar(name='neurotech', x= dtnp, y=list(df_topics_ym_count.neurotech),marker={'color': px.colors.qualitative.Prism[6]})])

# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_xaxes(type='category')
fig.update_layout(
            title={
            'text': 'Number news per topic' ,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Date",
            yaxis_title="N news stories" )
fig.update_xaxes(tickangle=90)
IMG_NAME = 'all_counts_bar_nodigitalxr.png'
PATH_IMG = PATH_MAIN + '/plots_raw/' + IMG_NAME
#fig.write_image(PATH_IMG)
fig.show()

In [14]:
dfvc = df_all.topic.value_counts().rename_axis('topic').to_frame('counts')
dfvc['topic'] = dfvc.index
dfvc

,counts,topic
topic,,
digitalxr,8115,digitalxr
climate,4749,climate
neurotech,688,neurotech


In [332]:
fig = px.bar(dfvc, x="topic", y="counts",
             color='topic',text='counts',color_discrete_map={'climate':px.colors.qualitative.Prism[4],
                                                            'digitalxr': px.colors.qualitative.Prism[1],
                                                            'neurotech': px.colors.qualitative.Prism[6]})
# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_xaxes(type='category')
fig.update_layout(
            title={
            'text': 'Number news per topic' ,
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Date",
            yaxis_title="N news stories" )
fig.update_xaxes(tickangle=0)
IMG_NAME = 'all_counts_bar_nodigitalxr.png'
PATH_IMG = PATH_MAIN + '/plots_raw/' + IMG_NAME
#fig.write_image(PATH_IMG)
fig.show()

In [15]:

dfcc = df_all.groupby(['country']).size().rename_axis('country').to_frame('counts')
dfcc['country'] = dfcc.index
dfcc.index = list(range(len(dfcc)))
dfccs = dfcc.sort_values(by='counts',ascending=False)

In [16]:
dfccs

,counts,country
12,3423,us
4,2091,fr
7,1879,it
5,1651,gb
2,1195,de
6,940,ie
9,617,ro
3,489,es
11,334,se
10,311,rs


In [391]:



fig = go.Figure(data=[
    go.Bar( x= list(dfcc['country']), y=list(dfcc['counts']),
           marker={'color': px.colors.qualitative.Prism[10]}, text=list(dfcc['counts']))])
   
# Change the bar mode

fig.update_layout(
            title={
            'text': 'Number news stories per country' ,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Country",
            yaxis_title="N news stories" )
fig.update_xaxes(tickangle=0)
IMG_NAME = 'all_counts_bar_nodigitalxr.png'
fig.show()

In [17]:
dfo = dfp.sort_values(by='n_outlets',ascending=False)
fig = go.Figure(data=[
    go.Bar( x= list(dfo['country']), y= list(dfo['n_outlets']),
           marker={'color': px.colors.qualitative.Prism[10]}, text= list(dfo['n_outlets']))])
   
# Change the bar mode

fig.update_layout(
            title={
            'text': 'Number of different outlet per country' ,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Country",
            yaxis_title="N outlets" )
fig.update_xaxes(tickangle=0)
IMG_NAME = 'all_counts_bar_nodigitalxr.png'
fig.show()

NameError: name 'dfp' is not defined

In [18]:
n_outlets=[]
n_news = []
c =[]
for i,d in df_all.groupby(['country']):
    c.append(i)
    n_outlets.append(int(len(list(set(d['domain'])))))
    n_news.append(int(len(d)))

In [19]:
dfp = pd.DataFrame(np.transpose([c, n_news, n_outlets]), columns=['country','n_news','n_outlets'])
dfp['n_news'] = dfp['n_news'].astype('int')
dfp['n_outlets'] = dfp['n_outlets'].astype('int')

In [406]:
fig = px.scatter(dfp, x="n_news", y="n_outlets", 
                 size='n_news', text='country')
fig.update_traces(textposition="bottom right")
fig.update_layout(
            title={
            'text': 'Number of news stories vs Number of different outlets per country' ,
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            yaxis_title="N different outlets",
            xaxis_title="N news stories" )
fig.update_xaxes(tickangle=0)

fig.show()

In [389]:
years = list(dfp['country'])
sr1 = list(dfp['n_news'])
sr2= list(dfp['n_outlets'])
#convert sr1
fig = go.Figure()
fig.add_trace(go.Bar(x=years, y=sr1,
                base=0,
                marker_color='rgb(158,202,225)',
                name='Revenue',
                marker_line_color='rgb(8,48,107)',
                marker_line_width=1.5,
                opacity= 0.7,
                text = sr1,
                textposition='outside'
                ))
fig.add_trace(go.Bar(x=years, y=sr2,
                base=sr3,
                marker_color='crimson',
                name='COGS<br>(Cost of Goods Sold)',
                marker_line_color='red',
                marker_line_width=1.5,
                opacity= 0.7,
                text = sr3,
                textposition='auto'
                ))
fig.update_layout(
    height=500,
    title_text='Apple Inc Revenue & Expense from 2017 to 2021'
    )
fig.show()

In [ ]:
fig = px.bar(dfvc, x="topic", y="counts",
             color='topic',text='counts',color_discrete_map={'climate':px.colors.qualitative.Prism[4],
                                                            'digitalxr': px.colors.qualitative.Prism[1],
                                                            'neurotech': px.colors.qualitative.Prism[6]})


In [503]:
df_all.shape

(13552, 16)

In [25]:
xdata = []
ydata = [c.upper() for c in countries]
for c in countries:
    dfc = df_all[df_all.country==c]
    xdata.append(list(df_all[df_all.country==c]['topic'].value_counts(normalize=True, sort=False).values*100))
    

In [508]:
#####

fig = go.Figure()
colors = [px.colors.qualitative.Prism[4], px.colors.qualitative.Prism[1],px.colors.qualitative.Prism[6]]

for i,t in enumerate(topics):
    for xd, yd in zip(xdata, ydata):
        fig.add_trace(go.Bar( 
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=120, r=10, t=140, b=80),
    showlegend=False,
)

annotations = []

for yd, xd in zip(ydata, xdata):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0]/2, y=yd,
                            text=str("%.1f"%xd[0]) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))

    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str("%.1f"%xd[i]) + '%',
                                    font=dict(family='Arial', size=12,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))

            space += xd[i]
fig.update_layout(
            title={
            'text': 'Percentage of news stories collected by technology',
            'y':0.85,
            'x':0.58,
            'xanchor': 'center',
            'yanchor': 'top'})
  
fig.update_layout(annotations=annotations)
fig.show()

In [20]:
raw_counts_climate = { 'at' :  { 'Afforestation':31,
                                'Reforestation':15,
                                 'Green hydrogen': 25,
                                'Carbon capture':8,
                                'Carbon storage':6,
                                'Climate engineering':7,
                                'Solar radiation management': 4,
                                'Sun shield': 3,
                                'Artificial photosynthesis':1,
                                'Geoengineering':6,
                                'Direct air capturing': 1},
                      'cz' : {'Green hydrogen': 25,
                             'Carbon storage': 23,
                              'Afforestation': 8,
                             'Carbon capture': 4,
                              'Sun shield': 3,
                             'Geoengineering':3,
                              'Cloud seeding':1,
                              'Carbon usage':1
                             },
                      'de' : { 'Afforestation' : 299,
                              'Green hydrogen' : 298, 
                              'Climate engineering':119,
                              'Cloud seeding':32,
                              'Geoengineering':7,
                              'Carbon storage': 7,
                              'Carbon sequestration' : 5, 
                              'Water splitting':2,
                              'Ocean fertilisation':1,
                              'Negative emission technology':1,
                              'Cloud whitening':1, 
                              'Artificial photosynthesis':2},
                      'es' :{ 'Afforestation':59,
                             'Carbon capture':10, 
                             'Carbon storage':6,
                             'Direct air capturing': 2,
                             'Sun shield': 12,
                             'Cloud seeding': 3,
                             'Carbon usage':2,
                             'Geoengineering': 1,
                             'Negative emission technology':1,
                             'Climate engineering': 1,
                             'Cloud modification':1},
                      
                      'fr':{ 'Green hydrogen': 432,
                            'Carbon storage': 293,
                            'Carbon capture': 26,
                            'Geoengineering':13,
                            'Cloud seeding': 10,
                            'Carbon usage':7,
                            'Sun shield':6,
                            'Direct air capturing':4,
                            'Solar radiation management':2,
                            'Ocean fertilisation':2,
                             },
                      
                      'gb':{ 'Carbon capture':228,
                           'Green hydrogen': 77,
                           'Carbon storage':55,
                           'Carbon sequestration':47,
                           'Afforestation': 14,
                           'Cloud seeding':12,
                           'Direct air capturing':9,
                           'Geoengineering': 7,
                           'Negative emission technology':4,
                           'Sun shield': 4,
                           'Climate engineering': 3, 
                           'Solar radiation management': 3, 
                           'Water splitting': 2,
                           'Carbon usage':3,
                           'Greenhouse gas removal': 1,
                           },
                      
                      'ie' : { 'Carbon capture':90 ,
                              'Green hydrogen': 59,
                              'Afforestation': 53,
                              'Carbon sequestration': 43,
                              'Carbon storage':28,
                              'Carbon usage' : 8,
                              'Cloud seeding': 4 ,
                              'Direct air capturing': 3,
                              'Sun shield': 2,
                              'Geoengineering': 2,
                              'Water splitting': 1 ,
                              'Negative emission technology': 1,
                              'Reforestation': 1,
                              'Climate engineering': 1},
                      
                      'it': {'Green hydrogen': 279,
                            'Reforestation': 40,
                            'Carbon storage': 29,
                            'Carbon capture': 22,
                            'Geoengineering' : 9,
                            'Carbon usage': 9,
                            'Water splitting': 4,
                            'Artificial photosynthesis': 3,
                            'Direct air capturing': 2,
                            'Ocean fertilisation': 1,
                            'Cloud seeding': 1},
                      
                      'nl': { 'Green hydrogen': 24,
                              'Carbon storage': 9,
                             'Afforestation' : 4,
                             'Sun shield': 2, 
                             'Geoengineering': 4,
                             'Carbon capture': 1,
                             'Carbon sequestration':1},
                      'ro':{ 'Afforestation': 120,
                            'Reforestation':22,
                            'Cloud seeding': 7,
                            'Green hydrogen':6, 
                            'Geoengineering':2,
                            'Carbon usage':3,
                            'Greenhouse gas removal':1,
                            'Carbon capture': 1,
                            'Artificial photosynthesis':1},
                      
                      'rs':{ 'Afforestation':17,
                            'Green hydrogen':10,
                            'Carbon storage':4,
                            'Carbon capture':3,
                            'Carbon usage':3,
                            'Geoengineering':2,
                            'Artificial photosynthesis':1},
                      
                      'se': {'Carbon storage':30,
                      'Green hydrogen': 25,
                      'Geoengineering': 17,
                      'Carbon capture': 11,
                      'Afforestation': 5,
                      'Cloud seeding':2},
                      
                      'us': { 'Carbon capture': 673,
                             'Green hydrogen': 574,
                             'Carbon sequestration': 267,
                             'Carbon storage': 130,
                             'Direct air capturing': 67,
                             'Cloud seeding': 36,
                             'Sun shield': 27,
                             'Afforestation': 16,
                             'Geoengineering': 14,
                             'Climate engineering': 13,
                             'Carbon usage': 10,
                             'Negative emission technology': 8,
                             'Greenhouse gas removal': 8,
                             'Solar radiation management': 8,
                             'Artificial photosynthesis': 5, 
                             'Water splitting':5,
                             'Artificial upwelling': 5,
                             'Reforestation':1,
                             'Cloud modification': 1
                          
                      }
                      
             
                        }

In [21]:
raw_counts_digitalxr = { 
    
    'at':{ 'Virtual reality': 41,
          'Virtual world': 21,
          'Digital twin': 11,
          'Extended reality': 14,
          'Metaverse': 8,
          'Digital avatar': 4,
          'NLP': 3,
          'Mixed reality': 1,
          'Smart glasses': 1,
          'Virtual environment':1
        
    },
    
    
    'de':{
        'Augmented reality': 196,
        'Virtual reality': 154,
        'Virtual world': 122,
        'Virtual environment': 43,
        'Metaverse': 42,
        'Mixed reality': 18,
        'NLP': 15,
        'Smart glasses': 15,
        'Digital avatar': 11,
        'Digital twin': 5,
        'Extended reality': 4},
    
    'es':{
        'Virtual reality':168,
        'Virtual world': 116,
        'Augmented reality': 113,
        'Metaverse': 95,
        'Mixed reality': 9,
        'Extended reality': 14,
        'NLP':6,
        'Digital twin':7 ,
        'Digital avatar':2, 
    },
    
    'fr':{
        'Virtual reality': 707,
        'Augmented reality': 445,
        'Metaverse': 357,
        'Mixed reality': 40,
        'Extended reality':36,
        'Smart glasses': 19,
        'Digital avatar': 17,
        'Virtual environment': 15
    },
    
    'gb':{
        'Virtual reality': 578,
        'Augmented reality': 323,
        'Metaverse': 193,
        'Virtual world': 124,
        'Mixed reality': 76,
        'Extended reality': 75,
        'NLP': 50,
        'Smart glasses': 46,
        'Augmented reality': 46, 
        'Virtual environment': 21
    },
    
    'ie': {
        'Virtual reality': 358,
        'Augmented reality': 172,
        'Metaverse': 111,
        'Virtual world': 124,
        'Mixed reality': 22,
        'Extended reality': 15,
        'NLP': 37,
        'Smart glasses': 28,
        'Virtual environment': 26,
        'Digital twin': 11,
        'Digital avatar': 8},
    
    'it': {
        'Augmented reality': 526,
        'Virtual reality': 697,
        'Metaverse': 297,
        'Virtual world': 146,
        'Extended reality': 83,
        'Smart glasses': 44,
        'Mixed reality': 39,
        'Digital avatar': 26,
        'Digital twin': 23,
        'NLP': 12
    },
    
    'nl': {
        'Virtual reality': 65,
        'Augmented reality': 59,
        'Extended reality':17,
        'Metaverse': 15,
        'Virtual world':14,
        'NLP':6,
        'Virtual environment': 5,
        'Digital avatar':5,
        'Smart glasses': 4,
        'Mixed reality':3
        
    },
    
    'ro': {
        'Virtual reality': 178,
        'Augmented reality': 89,
        'Extended reality': 61,
        'Digital avatar': 59,
        'Metaverse': 53,
        'Smart glasses': 32,
        'NLP': 10,
        'Mixed reality': 3,
        'Virtual world':44,
        'Virtual environment': 35
    },
    
    'rs': {
        'Augmented reality': 173,
        'Virtual reality': 161,
        'Virtual world': 22,
        'Metaverse': 16,
        'Smart glasses': 8,
        'Virtual environment': 3,
        'NLP':4
    },
    
    'se':{
        
        'Virtual reality': 201,
        'Augmented reality': 20,
        'Metaverse': 16,
        'Virtual world': 5,
        'Extended reality': 3,
        'Digital twin': 2,
        'NLP': 1,
        'Digital avatar':1
    },
    
    'us': {
        'Virtual reality':812, 
        'Augmented reality': 454, 
        'Metaverse': 353,
        'Virtual world': 188,
        'NLP': 126,
        'Mixed reality': 86,
        'Smart glasses':79,
        'Extended reality':55,
        'Digital twin': 28,
        'Virtual environment': 25,
        'Digital avatar': 23
    },
    
        'cz': {'Virtual reality': 98,
          'Augmented reality': 22,
          'Virtual world': 17, 
          'Digital twin': 8, 
          'Metaverse': 10, 
          'Virtual environment': 7, 
          'Extended reality':6,
          'NLP' : 7, 
          'Digital avatar':2, 
          'Mixed reality':1},
    
    
}

In [22]:
raw_counts_neurotech = {
    'at': {'Thought control':3,
           'Cyborg': 2,
           'Artificial eye': 1,
           'Cognitive enhancement':1,
           'Brain chip': 1,
           'Brain computer interface':1,
           'Human machine interface':1,
           'Artificial human':1},
    
    'cz': {
        'Brain implant': 8, 
        'Artificial uterus': 3,
        'Artificial eye': 3, 
        'Brain chip': 1,
        'Cyborg':1,
        'Neurotechnology':1
    },
    
    'de' : {
        'Neurotechnology':18,
        'Thought control': 8,
        'Brain chip':10,
        'Brain computer interface':7,
        'Cyborg': 5,
        'Artificial human': 3,
        'Cognitive enhancement': 1,
        'Neural control':1,
        'Brain implant':2
    },
    
    
    'es' : {
        'Neurotechnology': 8,
        'Artificial eye': 2,
        'Cognitive enhancement': 1,
        'Brain computer interface':1,
        'Neural interface': 1, 
        'Artificial human': 1,
        'Artificial uterus':1,
        'Brain chip': 1,
        'Thought control': 1
    },
    
    
    'fr':{ 
        'Brain computer interface':23,
        'Cyborg': 14,
        'Brain implant':7,
        'Brain chip':6,
        'Neurotechnology':6,
        'Neuroprosthetics': 3,
        'Neural interface':3,
        'Artificial human': 2,
        'Artificial brain': 2,
        'Neuromorphic engineering': 1,
        'Brain to brain': 1,
        'Artificial organ':1,
        'Artificial uterus':1,
        'Thought control':1
    
    },
    
    
    'gb':{
        'Brain computer interface': 36,
        'Cyborg': 19,
        'Neurotechnology':17,
        'Brain implant': 15,
        'Neural interface': 9,
        'Neuroprosthetics':10,
        'Brain chip':8,
        'Artificial eye':8,
        'Neuromorphic engineering':4,
        'Cognitive enhancement':4,
        'Neural control':4,
        'Artificial human': 4,
        'Artificial organ':3,
        'Transcranial electrical stimulation': 1,
        'Artificial brain':1,
        'Thought control':1
    },
    
    'ie' : {
        'Brain chip': 10,
        'Brain implant':8,
        'Brain computer interface': 7,
        'Cyborg':6,
        'Neurotechnology':1,
        'Neuromorphic engineering': 1,
        'Cognitive enhancement':1,
        'Brain to brain':1,
        'Artificial eye':1
    },
    
    'it':{
        'Cyborg': 16,
        'Neural interface': 7,
        'Brain implant': 7,
        'Artificial organ':3,
        'Brain chip': 3,
        'Neuromorphic engineering':2,
        'Artificial brain':2,
        'Artificial uterus':2,
        'Artificial eye':2,
        'Neurotechnology':4,
        'Artificial human':1,
        'Cognitive enhancement':1
    },
    
    'nl':{'Cyborg':2, 'Brain implant':1},
    
    'ro':{
        'Cyborg':5,
        'Brain chip':4,
        'Neurotechnology':5,
        'Brain computer interaction':4,
        'Artificial uterus':3,
        'Cognitive enhancement':2,
        'Artificial eye':1,
        'Neural interface':1,
        'Transcranial electrical stimulation':1,
        'Artificial brain':1,
    },
    
    'rs':{
        'Brain to brain':3,
        'Brain implant':2,
        'Brain chip':2,
        'Cyborg':1,
        'Artificial human':1
    },
    
    'se': {
        'Cyborg': 7,
        'Thought control': 4, 
        'Artificial uterus':1
    },
    
    
    'us':{
        
        'Brain computer interface': 70,
        'Neurotechnology': 51,
        'Brain implant':43,
        'Neural interface':36,
        'Brain chip':36,
        'Cyborg': 27,
        'Neuromorphic engineering':25,
        'Neuroprosthetics': 8,
        'Artificial human': 8,
        'Thought control': 7,
        'Artificial brain':6,
        'Brain to brain': 5,
        'Cognitive enhancement':4,
        'Artificial uterus': 3,
 'Artificial eye':3,
 'Neural control': 2,
 'Transcranial electrical stimulation': 2,
 'Artificial synapsis': 1,
 'Cognitive enhancement': 1,
 'Artificial organ' :1
    }
    
    
    
    
}

In [894]:
c = 'es'#countries[12]
t = 'climate'
dftmp = df_all[df_all.country==c]
alias = aliases[c]
xx, yy, cc = plot_kw_matches_counts(dftmp, 'kw_matches', countries_dict[c]['name'], c, t, PATH_MAIN, nw=14,
                           orient='v',aliases_dict=alias,save=False, ethics =False)
  

In [80]:
# topic-specific params
raw_counts = raw_counts_neurotech
t = 'neurotech'
topics_col = {'climate':'Greens', 'digitalxr' : 'Blues', 'neurotech' : 'Oranges'}
t_col = topics_col[t]

In [81]:
# create header
header = []
for c in raw_counts.keys():
    w = list(raw_counts[c].keys())
    header = header + w
    
header = list(set(header))

# add same keys for each country 
for c in raw_counts.keys():
    for w in header:
        if w not in list(raw_counts[c].keys()):
            raw_counts[c][w] = 0
            
# create df rows
hmp = []
row_id = []
for c in raw_counts.keys():
    row = []
    row_id.append(c)
    denom = sum(raw_counts[c].values())
    for w in header:
        #row.append(raw_counts_digitalxr[c][w]/len(df_all[(df_all.country==c) & (df_all.topic=='digitalxr')]))
        # normalise counts
        row.append(raw_counts[c][w]/denom*100)
    hmp.append(row)

# transpose, add counts across countries, sort
hmpt = np.transpose(hmp)
hmp_df = pd.DataFrame(hmpt,columns=row_id)
hmp_df['counts'] = hmp_df.sum(axis=1)

hmp_df['kw'] = header
hmp_df = hmp_df.sort_values(by=['counts'], ascending=False)

# keep only most frequent to plot
hmp_plot = hmp_df[hmp_df['counts']>0.05].reset_index(drop=True)
kw = hmp_plot['kw']
#hmp_plot = hmp_plot[raw_counts_digitalxr.keys()].values

In [82]:

z_text = np.around(hmp_plot[countries].values, decimals=1) 

fig = px.imshow(z_text,
               
               labels=dict(y=t+ " keywords", color="News percentage"),
                y= hmp_plot['kw'],
                x= list(hmp_plot[countries].columns) , color_continuous_scale=t_col,
                aspect="auto",text_auto=True)


fig.update_xaxes(
        side ='top',
        tickangle = 0
        #,title_font = {"size": 20}e
        ,title_standoff = 1
)


fig.update_layout(
            title={
            'text': 'Number of news mentioning ' +t+ ' keywords per country (normalised)',
            'y':1.0,
            'x':0.55,
            'xanchor': 'center',
            'yanchor': 'top'})


fig.show()

In [471]:
hmp_df['counts_avg'] = hmp_df['counts']/13
hmp_df[['counts_avg','kw']]

,counts_avg,kw
14,0.013401,physical integrity
9,0.205677,illegal
3,0.727196,negative impact
6,0.761004,fundamental rights
10,0.825131,accountability
8,0.827514,fairness
18,0.935159,discrimination
15,1.100139,insecurity
5,1.291958,dignity
17,2.102935,bias


In [68]:
fig = px.box(hmp_df, x="counts_avg")
fig.show()

In [28]:
kw_stats = dict.fromkeys(header)

In [ ]:
x:100=g:len

In [483]:
100*ge/len(df_all[df_all.topic=='climate'])

11.939355653821858

In [48]:
top1 = []
for c in countries:
    
    top1c = hmp_df[[c,'kw']].sort_values(by=[c], ascending=False).reset_index(drop=True)['kw'][0]
    top1.append(top1c)
    print(c + ' ' + top1c)

at Thought control
cz Brain implant
de Neurotechnology
es Neurotechnology
fr Brain computer interface
gb Brain computer interface
ie Brain chip
it Cyborg
nl Cyborg
ro Cyborg
rs Brain to brain
se Cyborg
us Brain computer interface


In [469]:
Counter(top1).most_common()

[('Afforestation', 4),
 ('Green hydrogen', 4),
 ('Carbon capture', 3),
 ('Reforestation', 1),
 ('Carbon storage', 1)]

# Outlet type

In [504]:
outlets_dict = dict.fromkeys(countries)

In [505]:
PATH_MAIN ="/Users/ilariabonavitacorp/Library/CloudStorage/OneDrive-TrilateralResearchLtd/trilateralResearch/TechEthos/media_analysis/analysis/"
for c in countries:
    df_outlets_tmp = pd.read_excel(PATH_MAIN + c + '/data/outlets_annotation.xlsx')
    
    outlets_dict[c]=df_outlets_tmp.set_index('domain').T.to_dict()



In [506]:
df_all['dom_type'] = None
df_all['dom_topic'] = None
nonedom = []
for i, r in df_all.iterrows():
    domain = r['domain']
    c = r['country']
    try:
        df_all['dom_type'].iloc[i] = outlets_dict[c][domain]['type']
        df_all['dom_topic'].iloc[i] = outlets_dict[c][domain]['topic']
    except:
        nonedom.append(domain)
        print(c + ' ' + domain)
        df_all['dom_type'].iloc[i] = None
        df_all['dom_topic'].iloc[i] = None

/Users/ilariabonavitacorp/.conda/envs/newscrawl/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



de golem
de bondguide
de rtl
de spiegel
de bondguide
de bondguide
de bondguide
de golem
de golem
it milano
it bufale
it fcinternews
it fcinternews
it milano
it ilfattonisseno
it bufale
it calciomercato
nl rtvnoord
rs niskevesti
rs balkans
rs mojnovisad
rs balkans
rs standard
rs espreso
rs mondo


In [50]:
Counter(list(df_all[df_all.topic=='neurotech']['dom_type'])).most_common()

[('Unsure', 323),
 ('GENERAL_NEWS', 119),
 ('TV_RADIO', 57),
 ('TABLOID', 51),
 ('LOCAL', 42),
 ('Other', 27),
 ('FREESHEET', 23),
 ('MAGAZINE', 22),
 ('NEWS_AGENCY', 13),
 ('NEWS_AGGREGATOR', 8),
 ('OTHER', 2),
 (None, 1)]

In [54]:
57/len(df_all[df_all.topic=='neurotech'])*100

8.284883720930232

In [60]:
153/len(df_all[df_all.topic=='climate'])*100

3.2217308907138342

In [507]:
Counter(list(df_all[df_all.topic=='digitalxr']['dom_topic'])).most_common()

[('GENERAL', 3565),
 ('TECH', 3284),
 ('ECONOMY', 544),
 ('CULTURE', 289),
 ('Other', 259),
 ('SCIENCE', 146),
 (None, 17),
 ('Unsure', 10),
 (nan, 1)]

In [952]:
Counter(list(df_all[df_all.topic=='digitalxr']['dom_topic'])).most_common()

[('GENERAL', 2906),
 ('TECH', 2321),
 (None, 1871),
 ('ECONOMY', 364),
 ('CULTURE', 289),
 ('Other', 207),
 ('SCIENCE', 146),
 ('Unsure', 10),
 (nan, 1)]

# Neuralink

In [29]:
df_nt = df_all[df_all.topic=='neurotech'].reset_index(drop=True)

In [30]:
len(df_nt)

688

In [31]:
df_nt.country.value_counts()

us    251
gb    117
fr     58
it     49
de     48
ro     44
rs     28
cz     26
ie     25
es     15
se     14
at     10
nl      3
Name: country, dtype: int64

In [63]:
ids_em = []
tsl = []
for i,r in df_nt.iterrows():
    txt = str(r['text']) + ' ' + str(r['title'])
    try:
        if ('Elon Musk' in txt) or  ('Neuralink' in txt):
            ids_em.append(i)
        elif 'Ilon Mask' in txt:
            #print(r['country'])
            #print(r['title'])
            ids_em.append(i)
        elif'Musk' in txt:
            print(r['title'])
            ids_em.append(i)
    except:
        print('')
        #print(r['title'])

Facebook zeigt Armband für Gedankensteuerung - OE24
Das Leben ist eine große Leinwand. Wirf soviel Farbe wie ...
Hirnimplantat machts möglich: Gelähmter kann mit Gedankenkraft schreiben
Muskeln aufbauen: Selbsttest soll zeigen, welche Muskeln Sie schnell auftrainieren können
Sprache direkt aus dem Gehirn
USA: Forschern übersetzen Gehirnströme eines gelähmten Mannes in Worte


In [64]:
len(ids_em)

239

In [46]:
len(tsl)

4

In [65]:
(len(ids_em)+1)/len(df_nt)*100

34.883720930232556

In [300]:
len(set(df_nt_em['country']))

13

In [299]:
df_nt_em = df_nt.iloc[ids_em]
dd = df_nt_em.groupby([df_nt_em["date"].dt.to_period('D')]).size()

xs = list(dd.index.astype('datetime64[M]'))
ys = list(dd.values)

xs = [str(s)[0:10] for s in xs]

fig = go.Figure()
fig.add_trace(go.Scatter(name='climate', x= xs, y=ys, 
               marker={'color': px.colors.qualitative.Prism[1]}))
fig.update_xaxes(type='category')
fig.update_layout(
            title={
            'text': 'Number of news titles mentioning Facebook or Meta' ,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Date",
            yaxis_title="N news titles" )
fig.update_xaxes(tickangle=90,tickformat="%Y-%M-%D")

fig.show()


In [291]:
el = []

for i,r in df_nt_em[df_nt_em['ethics_matches']!='[]'].iterrows():
    ek = convert_cell_to_list(r['ethics_matches'])
    k = [str(w).lower().strip() for w in ek]
    k = [" ".join(a.split()) for a in k]
    el.append(k)

lat_list = [item for sublist in el for item in sublist]

In [293]:
Counter(lat_list).most_common()

[('', 13),
 ('société', 10),
 ('společnost', 9),
 ('gesellschaft', 9),
 ('legal', 8),
 ('würden', 7),
 ('verantwortung', 7),
 ('sicherheit', 6),
 ('würde', 6),
 ('law', 6),
 ('society', 5),
 ('ethic', 4),
 ('privacy', 4),
 ('società', 3),
 ('bezpečnost', 2),
 ('sécurité', 2),
 ('security', 2),
 ('bezbednost', 2),
 ('recht', 1),
 ('gesellschafter', 1),
 ('rechts', 1),
 ('sociedad', 1),
 ('securitatea', 1),
 ('confidențialitatea', 1),
 ('societatea', 1),
 ('pravda', 1),
 ('responsibility', 1)]

# Metaverse

In [98]:
df_dx = df_all[df_all.topic=='digitalxr'].reset_index(drop=True)

In [207]:
ids_meta = []
for i,r in df_dx.iterrows():
    try:
        if ('Meta' in r['title']) or ('Facebook' in r['title']) or ('Zuckerberg' in r['title']) :
            ids_meta.append(i)
    except:
        print(r['title'])

nan
nan
nan
nan


In [278]:
df_dx.iloc[ids_bt].groupby(['country']).size()

country
cz    1
de    1
es    1
fr    5
gb    3
it    4
nl    1
ro    1
rs    9
us    2
dtype: int64

In [275]:
len(df_dx.iloc[ids_bt])

28

In [279]:

for w in ['Google','Facebook','Apple','Instagram',
              'Tesla','Android','Snapchat','Twitter']:
    ids_bt = []
    for i,r in df_dx.iterrows():

        try:
            if (w in r['title']) :
                ids_bt.append(i)
        except:
            pass
    
    print(w)
    print(len(set(df_dx.iloc[ids_bt]['country'])))
    print(len(ids_bt)/len(df_dx)*100)

Google
13
2.4645717806531113
Facebook
13
9.31608133086876
Apple
13
5.804066543438078
Instagram
11
0.8379544054220579
Tesla
6
0.1725200246457178
Android
9
0.751694393099199
Snapchat
10
0.6284658040665434
Twitter
10
0.3450400492914356


In [208]:
df_dx_meta = df_dx.iloc[ids_meta].reset_index(drop=True)

In [210]:
print(df_dx_meta.shape)

(977, 18)


In [211]:
len(ids_meta)/len(df_dx)*100

12.039433148490449

In [215]:
dd = df_dx_meta.groupby([df_dx_meta["date"].dt.to_period('D')]).size()

xs = list(dd.index.astype('datetime64[M]'))
ys = list(dd.values)

xs = [str(s)[0:10] for s in xs]

fig = go.Figure()
fig.add_trace(go.Scatter(name='climate', x= xs, y=ys, 
               marker={'color': px.colors.qualitative.Prism[1]}))
fig.update_xaxes(type='category')
fig.update_layout(
            title={
            'text': 'Number of news titles mentioning Facebook or Meta' ,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Date",
            yaxis_title="N news titles" )
fig.update_xaxes(tickangle=90,tickformat="%Y-%M-%D")

fig.show()


In [245]:
len(df_dx_meta[df_dx_meta['ethics_matches']!='[]'])/len(df_dx_meta)

0.5087001023541453

In [257]:
el = []
for i,r in df_dx_meta[df_dx_meta['ethics_matches']!='[]'].iterrows():
    ek = convert_cell_to_list(r['ethics_matches'])
    k = [str(w).lower().strip() for w in ek]
    k = [" ".join(a.split()) for a in k]
    el.append(k)

lat_list = [item for sublist in el for item in sublist]

In [262]:
sec = 42+40+28+26+16+6+5+1

In [263]:
priv = 218+10+1+4

In [280]:
sec/len(df_dx_meta)*100

16.786079836233366

In [281]:
priv/len(df_dx_meta)*100

23.848515864892526

In [259]:
Counter(lat_list).most_common()

[('privacy', 218),
 ('società', 171),
 ('société', 109),
 ('', 47),
 ('droit', 44),
 ('sécurité', 42),
 ('law', 42),
 ('society', 40),
 ('security', 35),
 ('legal', 34),
 ('sicurezza', 28),
 ('würden', 27),
 ('vie priver', 27),
 ('justice', 27),
 ('seguridad', 26),
 ('sicherheit', 16),
 ('integrity', 16),
 ('responsibility', 14),
 ('sociedad', 13),
 ('legge', 13),
 ('verantwortung', 12),
 ('würde', 10),
 ('privacidad', 10),
 ('integritate', 10),
 ('lag', 10),
 ('společnost', 9),
 ('éthique', 8),
 ('juridique', 8),
 ('ethnic', 8),
 ('gesellschaft', 7),
 ('ley', 7),
 ('responsabilità', 7),
 ('responsabilidad', 6),
 ('responsabilité', 6),
 ('bias', 6),
 ('bezbednost', 6),
 ('datenschutz', 5),
 ('discrimination', 5),
 ('securitate', 5),
 ('zakon', 5),
 ('wohlbefinden', 4),
 ('bienestar', 4),
 ('eprivacy', 4),
 ('accountability', 4),
 ('responsabili', 4),
 ('sostenibilità', 4),
 ('recht', 3),
 ('droite', 3),
 ('impact négatif', 3),
 ('human right', 3),
 ('negative impact', 3),
 ('welzijn', 

In [ ]:
plot_kw_matches_counts(df_dx_meta, 'ethics_matches', country, country_code, topic, path, nw=14,
                           orient='v',aliases_dict={},save=True, ethics =False)

# Ethics

In [23]:
df_ethics = copy.deepcopy(df_all)
df_ethics = df_all[df_all.ethics_matches!='[]']
df_ethics = df_ethics[df_ethics.ethics_matches!='['']']
df_ethics = df_ethics[df_ethics.ethics_matches != "['']"].reset_index(drop=True)
print(df_ethics.shape)
print(len(df_ethics)/len(df_all))

(5037, 16)
0.37167945690672965


In [24]:
ethics_dict_climate = {
    'at': {
        'responsibility': 10,
     'sustainability': 6,
     'law': 5,
     'society': 4,
     'insecurity': 3,
     'well-being': 2,
     'ethics': 1,
     'negative impact': 1,
      'dignity': 1},
    
        'cz':{
         'security': 6,
      'ethics': 6,
 'sustainability': 2,
 'law': 2,
 'legal': 2,
 'responsibility': 2,
 'well-being': 1,
 'negative impact': 1
    },
    
    'de' : {'society': 69,
 'sustainability': 46,
 'responsibility': 43,
 'security': 38,
 'law': 32,
 'dignity': 18,
 'insecurity': 16,
 'justice': 9,
 'privacy': 6,
 'injustice': 6,
 'human rights': 5,
 'discrimination': 3,
 'negative impact': 1},
    
    'es':{ 'law': 18,
 'security': 13,
 'responsibility': 10,
 'society': 6,
 'justice': 6,
 'sustainability': 5,
 'human rights': 5,
 'illegal': 5,
 'well-being': 3,
 'legal': 3,
 'dignity': 3,
 'negative impact': 1},
    
    'fr': {'society': 175,
 'law': 118,
 'security': 60,
 'responsibility': 26,
 'justice': 21,
 'negative impact': 17,
 'legal': 13,
 'fundamental right': 6,
 'fairness': 5,
 'ethics': 3,
 'human rights': 2,
 'well-being': 1,
 'sustainability': 1},
    
    'gb' : {'privacy': 56,
 'society': 48,
 'sustainability': 47,
 'responsibility': 30,
 'law': 29,
 'legal': 27,
 'security': 26,
 'justice': 24,
 'human rights': 14,
 'integrity': 7,
 'negative impact': 6,
 'ethics': 4,
 'accountability': 3,
 'physical integrity': 1,
 'fairness': 1,
 'dignity': 1},
    
'ie' : {'society': 44,
 'law': 35,
 'sustainability': 24,
 'responsibility': 21,
 'legal': 20,
 'security': 18,
 'privacy': 12,
 'justice': 12,
 'negative impact': 4,
 'integrity': 4,
 'accountability': 2,
 'bias': 2,
 'discrimination': 2,
 'human rights': 2,
 'vulnerable groups': 1,
 'dignity': 1,
 'well-being': 1,
 'ethics': 1},
    'it' : {'society': 142,
 'sustainability': 103,
 'security': 44,
 'law': 35,
 'responsibility': 25,
 'well-being': 17,
 'justice': 10,
 'integrity': 4,
 'ethics': 3,
 'human rights': 3,
 'legal': 2,
 'privacy': 1,
 'dignity': 1,
 'fairness': 1,
 'discrimination': 1},
    
    'nl': { 'sustainability': 2,
 'privacy': 2,
 'responsibility': 1,
 'society': 1,
 'law': 1,
 'legal': 1,
 'security': 1},
    
    'ro' :  {'security': 8,
 'law': 8,
 'society': 6,
 'legal': 5,
 'responsibility': 4,
 'discrimination': 2,
 'fairness': 2,
 'well-being': 1,
 'insecurity': 1,
 'justice': 1,
 'integrity': 1,
 'human rights': 1},
    
    
    'rs': {'law': 20,
 'responsibility': 11,
 'society': 7,
 'sustainability': 6,
 'legal': 4,
 'security': 4,
 'negative impact': 3,
 'justice': 3,
 'integrity': 1,
 'dignity': 1,
 'ethics': 1,
 'human rights': 1,
 'fairness': 1,
 'bias': 1,
 'well-being': 1},
    
    'se': { 'society': 34,
 'responsibility': 28,
 'security': 11,
 'sustainability': 10,
 'justice': 8,
 'dignity': 3,
 'ethics': 2,
 'human rights': 2},
    
    'us':{'sustainability': 181,
 'law': 172,
 'security': 137,
 'society': 112,
 'responsibility': 85,
 'justice': 59,
 'legal': 45,
 'accountability': 21,
 'ethics': 18,
 'integrity': 17,
 'human rights': 17,
 'negative impact': 16,
 'privacy': 6,
 'bias': 6,
 'fairness': 3,
 'discrimination': 3,
 'vulnerable groups': 2,
 'dignity': 1}
    
    
}


In [25]:
ethics_dict_digitalxr = {
    'at':{
        'society': 9,
 'security': 3,
 'sustainability': 2,
 'privacy': 2,
 'insecurity': 2,
 'responsibility': 2,
 'well-being': 1
    },
    

    
     'cz': {'security': 8,
 'legal': 3,
 'responsibility': 2,
 'sustainability': 1,
 'bias': 1},
    
    
    'de':{'security': 35,
 'privacy': 34,
 'society': 16,
 'law': 16,
 'responsibility': 15,
 'sustainability': 12,
 'well-being': 7,
 'dignity': 3,
 'insecurity': 2,
 'fairness': 1,
 'negative impact': 1},
    
    
'es' : { 'society': 80,
 'security': 65,
 'law': 35,
 'privacy': 33,
 'responsibility': 22,
 'well-being': 19,
 'legal': 16,
 'justice': 13,
 'ethics': 13,
 'sustainability': 8,
 'human rights': 7,
 'dignity': 6,
 'illegal': 5,
 'bias': 4,
 'discrimination': 4,
 'negative impact': 3,
 'integrity': 2,
 'fundamental rights': 2,
 'human rights': 2,
 'fairness': 1},
    
    'fr':{
        'society': 295,
 'law': 220,
 'security': 111,
 'privacy': 33,
 'justice': 19,
 'legal' : 13,
 'responsibility': 28,
 'ethics': 19,
 'bias': 7,
 'discrimination': 5,
 'well-being': 4,
  'integrity': 3,
 'negative impact': 3,
 'dignity': 2,
 'fairness': 1,
 'physical integrity': 1},
    
'gb' :{'privacy': 181,
 'society': 76,
 'security': 54,
 'law': 52,
 'responsibility': 38,
 'sustainability': 32,
 'justice': 30,
 'legal': 27,
 'bias': 16,
 'ethics': 14,
 'integrity': 14,
 'human rights': 11,
 'negative impact': 6,
 'dignity': 4,
 'fairness': 3,
 'well-being': 3,
 'accountability': 3,
 'discrimination': 3},
    
'ie' : {'privacy': 96,
 'society': 56,
 'security': 40,
 'law': 31,
 'responsibility': 21,
 'sustainability': 18,
 'legal': 18,
 'ethics': 15,
 'bias': 11,
 'justice': 8,
 'integrity': 7,
 'discrimination': 5,
 'negative impact': 5,
 'accountability': 4,
 'fairness': 3,
 'human rights': 2,
 'dignity': 1},
    
    'it' : {'society': 294,
 'security': 153,
 'sustainability': 70,
 'law': 69,
 'privacy': 67,
 'responsibility': 44,
 'well-being': 37,
 'justice': 12,
 'legal': 12,
 'ethics': 8,
 'dignity': 4,
 'discrimination': 2,
 'human rights': 2,
 'negative impact': 2,
 'integrity': 1,
 'discrimination': 1,
 'bias': 1},
    

    'nl': {  'privacy': 15,
 'responsibility': 5,
 'society': 4,
 'law': 4,
 'human rights': 4,
 'security': 3,
 'well-being': 3,
 'fundamental rights': 3,
 'sustainability': 2,
 'discrimination': 1,
 'dignity': 1,
 'fairness': 1,
 'justice': 1},
           
 'ro': {'security': 25,
 'society': 22,
 'privacy': 6,
 'legal': 6,
 'integrity': 5,
 'law': 4,
 'accountability': 4,
 'well-being': 3,
 'human rights': 2,
 'discrimination': 1,
 'sustainability': 1,
 'fairness': 1},
           
'rs' : {'law': 26,
 'security': 25,
 'society': 10,
 'justice': 10,
 'responsibility': 9,
 'privacy': 7,
 'legal': 6,
 'human rights': 6,
 'ethics': 6,
 'integrity': 4,
 'sustainability': 2,
 'bias': 2,
 'well-being': 2,
 'dignity': 1},
           
'se' : { 'society': 17,
 'security': 12,
 'ethics': 10,
 'responsibility': 5,
 'dignity': 4,
 'sustainability': 3,
 'human rights': 2,
 'well-being': 2,
 'bias': 2,
 'legal': 1,
 'justice': 1},
           
           'us': {'security': 142,
 'privacy': 80,
 'law': 70,
 'society': 69,
 'justice': 43,
 'legal': 40,
 'bias': 33,
 'responsibility': 23,
 'integrity': 19,
 'ethics': 19,
 'sustainability': 15,
 'fairness': 10,
 'negative impact': 9,
 'human rights': 9,
 'accountability': 8,
 'discrimination': 6,
 'dignity': 2,
 'well-being': 1}
    
    
}

In [26]:
ethics_dict_neurotech = {
    
    'at' : {
        'responsibility': 2, 'law': 2, 'privacy': 1, 'society': 1
    },
    
    'cz' : {'safety': 3, 'ethics': 2, 'legal': 1},
    
    'de' : {'security': 11,
 'society': 11,
 'responsibility': 9,
 'justice': 3,
 'privacy': 2,
 'law': 1,
 'human rights': 1},
  
  'es': { 'society': 4,
 'law': 3,
 'discrimination': 2,
 'security': 2,
 'legal': 2,
 'privacy': 2,
 'ethics': 2,
 'integrity': 1,
 'physical integrity': 1,
 'illegal': 1,
 'bias': 1,
 'justice': 1,
 'human rights': 1},
  
  'fr':{ 'society': 13,
 'law': 10,
 'ethics': 5,
 'security': 4,
 'privacy': 3,
 'physical integrity': 2,
 'integrity': 2},
  
  'gb' : {'privacy': 12,
 'society': 10,
 'ethics': 3,
 'legal': 3,
 'security': 3,
 'dignity': 3,
 'law': 3,
 'bias': 2,
 'responsibility': 1},
  
  'ie': {'privacy': 3,
 'security': 3,
 'society': 3,
 'sustainability': 1,
 'negative impact': 1},
  
  
  'it': {'society': 13,
 'ethics': 4,
 'privacy': 4,
 'law': 4,
 'responsibility': 2,
 'well-being': 2,
 'security': 2,
 'bias': 1,
 'discrimination': 1,
 'justice': 1,
 'legal': 1,
 'human rights': 1,
 'integrity': 1},
  
  'nl':{'privacy':1},
  
  'ro':{'society': 6, 'security': 5, 'law': 2, 'privacy': 1},
  
  'rs':{'law': 4,
 'security': 3,
 'responsibility': 3,
 'justice': 2,
 'society': 2,
 'sustainability': 1,
 'ethics': 1},
  
  'se':{ 'integrity': 3,
 'security': 2,
 'society': 2,
 'ethics': 2,
 'responsibility': 1,
 'justice': 1},
  
  'us': {'security': 23,
 'privacy': 21,
 'law': 20,
 'society': 18,
 'ethics': 10,
 'legal': 8,
 'responsibility': 8,
 'bias': 7,
 'integrity': 5,
 'human rights': 5,
 'justice': 5,
 'fairness': 1,
 'negative impact': 1,
 'accountability': 1,
 'dignity': 1}
  
  
  
  
    
}

In [54]:
for i, r in df_all[df_all.topic=='neurotech'].iterrows():
    if isinstance(r['text'],str):
        if 'Chile' in r['text'] or 'Chili' in r['text'] or 'Cile' in r['text']  or 'Čile' in r['text'] :
            print(r['country'])
            print(r['title'])

es
Chile, laboratorio mundial de los neuroderechos
es
La ciencia ya lee tu cerebro, pronto desvelará hasta tu subconsciente
fr
Chili: vote d'une loi protégeant les "neurodroits" ou droits du cerveau
fr
Chili: vote d'une loi protégeant les "neurodroits" ou droits du cerveau
gb
Cyborg skin, flying cars and THAT boat: Here’s our favourite scientific images from March
us
The Movement to Protect Your Mind From Brain-Computer Technologies
us
'This is not science fiction,' say scientists pushing for 'neuro-rights'


In [55]:
df_all.shape

(13552, 16)

In [56]:
ust=3423
ust/13552

0.25258264462809915

In [38]:
# topic-specific params
t = 'digitalxr'
raw_counts = ethics_dict_digitalxr
topics_col = {'climate':'Greens', 'digitalxr' : 'Blues', 'neurotech' : 'Oranges'}
t_col = topics_col[t]

In [39]:
# create header
header = []
for c in raw_counts.keys():
    w = list(raw_counts[c].keys())
    header = header + w
    
header = list(set(header))

# add same keys for each country 
for c in raw_counts.keys():
    for w in header:
        if w not in list(raw_counts[c].keys()):
            raw_counts[c][w] = 0
            
# create df rows
hmp = []
row_id = []
for c in raw_counts.keys():
    row = []
    row_id.append(c)
    denom = sum(raw_counts[c].values())
    for w in header:
        #row.append(raw_counts[c][w]/len(df_all[(df_all.country==c) & (df_all.topic==t)]))
        #row.append(raw_counts[c][w]/len(df_ethics[(df_ethics.country==c) & (df_ethics.topic==t)])*100)
        # normalise counts
        row.append(raw_counts[c][w]/denom*100)
    hmp.append(row)

# transpose, add counts across countries, sort
hmpt = np.transpose(hmp)
hmp_df = pd.DataFrame(hmpt,columns=row_id)
hmp_df['counts'] = hmp_df.sum(axis=1)

hmp_df['kw'] = header
hmp_df = hmp_df.sort_values(by=['counts'],ascending=False)

# keep only most frequent to plot
hmp_plot = hmp_df[hmp_df['counts']>0.05].reset_index(drop=True)
kw = hmp_plot['kw']
#hmp_plot = hmp_plot[raw_counts_digitalxr.keys()].values

In [437]:
 mp = np.mean(hmp_plot[countries].mean(axis=0))

In [40]:
z_text = np.around(hmp_plot[countries].values, decimals=1) 

fig = px.imshow(z_text,
               
               labels=dict(y=t+ " keywords", color="News percentage"),
                y= hmp_plot['kw'],
                x= list(hmp_plot[countries].columns) , color_continuous_scale='Greys',
                aspect="auto",text_auto=True)


fig.update_xaxes(
        side ='top',
        tickangle = 0
        ,title_font = {"size": 95}
        #,title_standoff = 0.1
)


fig.update_layout(
            title={
            'text': 'Number of news mentioning ELSI-keywords, per country (normalised) - ' + t,
            'y':0.98,
            'x':0.50,
            'xanchor': 'center',
            'yanchor': 'top'})


fig.show()

In [41]:
hmp_plot['at']

0     14.285714
1     42.857143
2      9.523810
3      0.000000
4      9.523810
5      9.523810
6      0.000000
7      0.000000
8      4.761905
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
14     9.523810
15     0.000000
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.000000
21     0.000000
Name: at, dtype: float64

In [37]:
z_text.sum(axis=0)

array([100. , 100. ,  99.9,  99.7,  99.9, 100. , 100.1,  99.9, 100. ,
       100. , 100. , 100.1,  99.8])

In [467]:
fig = px.imshow(hmp_plot[countries],
               
               labels=dict(x="Country", y=t+ " keywords", color="News percentage"),
                y= hmp_plot['kw'],
                x= list(hmp_plot[countries].columns) , color_continuous_scale='Viridis',
                aspect="auto",text_)
               
fig.update_xaxes(
        side ='bottom',
        tickangle = 0
        #,title_font = {"size": 20}e
        #,title_standoff = 25
)

fig.update_layout(
            title={
            'text': 'Number of news mentioning ELSI-keywords, per country (normalised) - ' + t,
            'y':0.95,
            'x':0.50,
            'xanchor': 'center',
            'yanchor': 'top'})
  

fig.show()

In [500]:
hmp_df['counts_avg'] = hmp_df['counts']/13
hmp_df[['counts_avg','kw']]

,counts_avg,kw
7,0.106838,fairness
9,0.106838,accountability
0,0.699301,well-being
5,0.827991,dignity
3,0.961538,negative impact
8,0.961538,illegal
14,1.442308,safety
13,1.492042,physical integrity
11,1.709402,sustainability
19,2.141235,human rights


In [501]:
hmp_df_s = hmp_df.sort_values(by='counts_avg', ascending=False)

In [502]:
fig = px.bar(hmp_df_s, x='kw', y='counts_avg', color_discrete_sequence= ['grey']*len(hmp_df_s))

fig.update_layout(
            title={
            'text': 'Average percentage of news mentioning ELSI-keywords (' +t+ ')',
            'y':0.95,
            'x':0.50,
            'xanchor': 'center',
            'yanchor': 'top',
            },
            xaxis_title="ELSI-keyword",
            yaxis_title="Average percentage of news",
            yaxis_range = [0,35])

fig.show()

In [496]:
etlen = []
t='neurotech'
for c in countries:
    print(c)
    av = len(df_ethics[(df_ethics.country==c)&(df_ethics.topic==t)])/len(df_all[(df_all.country==c)&(df_all.topic==t)])
    print(av)
    etlen.append(av)
    

at
0.3
cz
0.6153846153846154
de
0.5416666666666666
es
0.5333333333333333
fr
0.5
gb
0.27350427350427353
ie
0.36
it
0.4489795918367347
nl
0.3333333333333333
ro
0.20454545454545456
rs
0.32142857142857145
se
0.7142857142857143
us
0.2868525896414343


In [497]:
np.mean(etlen)

0.417947241843087

In [257]:
dict(cc)

{'security': 23,
 'privacy': 21,
 'law': 20,
 'society': 18,
 'ethic': 10,
 'legal': 8,
 'responsibility': 8,
 'bias': 7,
 'integrity': 5,
 'human rights': 5,
 'justice': 5,
 'fairness': 1,
 'nagative impact': 1,
 'accountability': 1,
 'dignity': 1}

In [315]:
en_al = { 'negative impact': ['human might', 'negative impact','negatively impact'],
         'human rights':['human right'],'responsibility':['irresponsibility'],
 'ethics': ['ethnic', 'ethic'], 'privacy':['eprivacy'],
         'vulnerable groups':['vulnerable group','vulnerable gop']}

In [415]:
ethics_dict_climate['ro']

{'security': 8,
 'law': 8,
 'society': 6,
 'legal': 5,
 'responsibility': 4,
 'discrimination': 2,
 'fairness': 2,
 'well-being': 1,
 'insecurity': 1,
 'justice': 1,
 'integrity': 1,
 'human rights': 1,
 'ethics': 0,
 'negative impact': 0,
 'dignity': 0,
 'illegal': 0,
 'accountability': 0,
 'sustainability': 0,
 'physical integrity': 0,
 'bias': 0,
 'fundamental right': 0,
 'injustice': 0,
 'vulnerable groups': 0,
 'privacy': 0}

In [414]:
c = 'ro'#countries[12]
t = 'climate'
dftmp = df_ethics[(df_ethics.country==c)&(df_ethics.topic==t)]
alias = aliases[c]
xx, yy, cc = plot_kw_matches_counts(dftmp, 'ethics_matches', countries_dict[c]['name'], c, t, PATH_MAIN, nw=14,
                           orient='v',aliases_dict=ro_al,save=False, ethics =False)

In [223]:
ro_al = {'securitate':['securitatea'],'societate':['societatea'],'responsabilitate':['responsabilitatea'],
         'integritate':['integritatea'],'confidențialitatea':['confidențialitate','confidentialitate'],
         'juridic':['juridice','juridică'],'justiție':['justiției'], 'bunăstarea' :['bunăstare']}

In [113]:
de_al = {'nachhaltigkeit':['nachhaltigkeits'], 'verantwortung':['verantwortungs'],
       'gesellschaft':['gesellschafts','gesellschafter'],'recht':['rechte'],
       'menschenrechte':['menschenrechten']}

In [125]:
es_al = {'responsabilidad':['corresponsabilidad'],'derechos humanos':['derecho humano']}

In [154]:
fr_al = {'droit' : ['droite'], 'préjugé':['préjugés'], 'juridique':['juridiquer'],
         'bien l être':['bien d être','bien c être'],'éthique':['éthiquer']}

# Functions

In [208]:
aliases = { 'at': {"grün wasserstoff":["grüner wasserstoff","grünen wasserstoff","grünem wasserstoff",
                                       "grünen wasserstoffs",
                                       'in wasserstoff',"grüne wasserstoff","grüner   wasserstoff","grün   wasserstoff"],
                   "aufforstung" : ["aufforstungen",'bewaldung','aufforsten'],
                   'sonnenblende':['sonnenblenden'],
                   "metaversum" : ["metaversums", "metaverse",'virtueller welt'],
                   "virtuelle realität" : ["virtual reality","virtualreality",
                                           "virtueller realität","virtuellen realität",
                                           'virtuelle realitäten','virtuellen realitäten','vr','virtuel reality'],
                   "virtuelle welt" : ['virtuellen welt','virtueller welt','virtuellen welt','virtuellwelt'],
                   "erweitern realität" : ['augmented reality','augmented realitiy', 'erweitert realität',
                                           'augmentedreality','erweiterten realität','erweiterter realität',
                                           'erweiterte realität'],

                   "virtuelle umgebung" : ['virtueller umgebung','virtuellen umgebung','virtuelle umgebungen','virtueller umgebungen'],
                   'digital zwilling':['digitale zwillinge',
                                       'digitalen zwillings','digitaler zwilling','digitale zwilling'],
                   'digitaler avatar':['digitale avatare','digital avatare'],
                   'cyborg':['cyborgs'],
                   'gehirnchip':['gehirnchips'],
                   'hirncomputerschnittstelle':['hirncomputerschnittstellen'],
                   'gehirncomputerschnittstellen':['gehirncomputerschnittstelle'],
                   'würde':['würden','würnde'],
                   'rechts':['rechte','recht'],
                   'gesellschaft':['gesellschafter','gesellschaften'],
                   'datenschutz':['datenschutzes'],
                   'sicherheit' : ['sicherheits'],
                   'cloud seeding' : ['cloudseeding','wolkenbildung'],
                   'geoengineering':['geoengineerings','geo engineering'],
                   'klimatechnik' :['klimatechniker'],'gedankensteuerung' : ['steuerung'],
                   'climate engineering':['engineering','klimaengineering'],
                   'co2 abscheidung' : ['von treibhausgasen','co2 abscheiden','co2 abescheidung'],
                   'Recht':['rechts'], 'Würde':['würde'] , 
                   'carbon capture': ['kohlenstoffbindung'],
                   "kohlenstoffspeicherung": ['kohlenstoffspeicher']},


           'cz':{'virtuální realita':['vr','virtual reality'], 'nlp':['zpracování přirozeného jazyka'],
                 'metaverse' :['fiktivní vesmír'],'digitální avatar':['virtuální avatar']},


           'es': {'almacenamiento de carbono' : ['del carbono'],
                  'captura de carbono' : ['del dióxido de carbono'],
                  'metaverso' : ['metaverso él','metaver él', 'metaver','metauniverso','metaverso','metaversos','metaverse'],
                  'neurotecnología': ['neurotecnologías'],
                  'Interfaz Cerebro Máquina' : ['Interfaz Cerebro Máquín'],
                  'derechos humanos' : ['derecho humano'], 'siembra de nube' : ['sembrar nube'],
                  'uso de carbón' : ['uso del carbón'],
                  'realidad virtual' : ['rv'],
                  'mundo virtual': ['entorno virtual','submundo virtual','mundo no virtual','mundovirtual','mundo virutal',
                                    'entornos virtuales'],
                  'realidad aumentada':['realidad aumentenda','realidad aumentado','realidad aumentadar','surrealidad aumentado'],
                  'realidad extendida':['realidad extendido','er','realidades extendidas','realidad extensivo','realidad extendido'],
                  'realidad mixta':['realidad mixto'],'gemelo digital':['gemelar digital'],
                  'interfaz cerebro máquina':['interfaz cerebro máquín']


                 },


           'fr':{'hydrogèn vert':['hydrogène vert','hydrogène   vert','hydrogéner vert','hydrogène    vert','hydrogèn vert','hydrogèn    vert','hydrogèn   vert'],
                 'captage de carbone':['captage de carbone','capture de carbone','captur de carbone','capter le carbone',
                                       'capture de dioxyde de carbone','capteur de carbone','capter de carbone','captage de dioxyde de carbone'],

                 'puits de carbone' : ['puits de   carbone','puits   de carbone','puit de carbone','puits à carbone','stockage en carbone','stockage de carbone',
                                       'stocker de le carbone','stocker le carbone','puits carbone','puits de    carbone','stocker de carbone'],
                 'utilisation de carbone': ['utilisation de charbon','réutilisation de carbone'],
                 'bouclier solaire':['parasol spatial'],
                 'fertilisation de l océan':['fertilisation    de océan'],
                 'géo ingénierie': ['géo ingénieri'],

                 'métaverse':['métavers','métaver','metaver',],
                 'réalité virtuel':['vr','réaliter virtuel','réalité virtuelle','réalite virtuel'],
                 'réalité augmentée':['réalité augmenter','réaliter augmenter'],
                 'monde virtuel':['monde virtuels'],
                 'réalité mixte':['réalité mixée','réalité mix'],
                 'environnement virtuel':['environnement virtualiser'],
                 'réalité étendu':['xr'],
                 'avatar virtuel':['avatar virtuels'],
                 'nlp': ['traitement automatique de langage naturel','traitement automatique de langage'],
                 'interface cerveau machine':['interfac cerveau machine','interfacer cerveau machine'],
  'interface neuronal':['interfacer neuronal']  ,'cyborg':['cyborgs']
                 
                 
                },


           'gb' : {'carbon capture': ['can capture','carbon   capture','carbon capturing','carbon be capture'], 'green hydrogen':['green   hydrogen'],
                   'carbon storage': ['carbon store','carbon   store'],'cloud seeding':['cloud seed'],
                   'negative impact':['human might','negative impact'], 'ethics' : ['ethnic','ethic'],
                   'carbon sequestration':['carbon sequestering'], 'carbon usage':['carbon use','carbone usage'],
                   'sun shield':['sunshield'],
                   'virtual reality':['vr'],
                   'nlp':['natural language processing'],
                   'virtual world':['virtual 3d world'],
                   'extended reality':['xr','extend reality'],
                   'smart glasses':['smart ar glass','smartglasse','smart sunglass','smart eyeglass'],
                   'metaverse':['metaverses'],
                   'mixed reality':['mix reality'],
                   'virtual environment':['virtual environments'],'augmented reality':['augment reality'],
                   
                   'brain computer interface':['brain machine interface','brain computer interfacing'],
    'neuroprosthetics':['neural prosthesis'],
        'cyborg':['cyborgs'],'neuroprosthetics':['neuroprosthetic','neural prosthesis'],
            'brain implant':['brain implants'],
                'neuromorphic engineering':['neuromorphic computing'],
    'neurotechnology':['neurotech'],'neural control':['neurally control']
                  },

           'ie' : {'carbon capture': ['can capture','carbon   capture','carbon capturing','carbon be capture','carbon gas capture'], 'green hydrogen':['green   hydrogen'],
                   'carbon storage': ['carbon store','carbon   store'],'cloud seeding':['cloud seed'],
                   'negative impact':['human might','negative impact'], 'ethics' : ['ethnic','ethic'],
                   'carbon sequestration':['carbon sequestering'], 'carbon usage':['carbon use','carbone usage'],
                   'sun shield':['sunshield'],

                   'virtual reality':['vr'],
                   'nlp':['natural language processing'],
                   'virtual world':['virtual 3d world'],
                   'extended reality':['xr','extend reality'],
                   'smart glasses':['smart ar glass','smartglasse','smart sunglass','smart eyeglass','smart glass','smart glass'],
                   'metaverse':['metaverses','meataverse'],
                   'mixed reality':['mix reality'],
                   'virtual environment':['virtual environments'], 'augmented reality':['augment reality'],
                   
                                      'brain computer interface':['brain machine interface','brain computer interfacing'],
    'neuroprosthetics':['neural prosthesis'],
        'cyborg':['cyborgs'],'neuroprosthetics':['neuroprosthetic','neural prosthesis'],
            'brain implant':['brain implants'],
                'neuromorphic engineering':['neuromorphic computing'],
    'neurotechnology':['neurotech'],'neural control':['neurally control']
                  },

           'it' :{ "stoccaggio del carbonio":["stoccaggio di carbonio"],
                  "(r)imboschimento" : ["imboschimento","imboschimenti","rimboschimento","rimboschimenti"],
                  "geoingegneria" : ["geo ingegneria", "geo-ingegneria"],
                  "idrogeno verde" : ["idrogeno verde2","indrogeno verde"],
                  'cattura del carbonio':['cattura carbonio'],
                  'realtà virtuale' : ['vr','realtà virtuale e aumentata','realtà virtuale aumentata',
                                       'ar vr','vr ar','realtà aumentata virtuale','realta virtuale','realtà aumentata o virtuale',
                                       'realtà virtuale10','realtà virtuale9','realtà aumentata e virtuale',
                                       'realtà virtuale aumentata','realta aumentata virtuale',
                                       'realtà virtuale ed aumentata'],
                  'realtà aumentata' :['ar','realtà virtuale e aumentata','realtà virtuale aumentata','realtà virtuali',
                                       'realtà aumentate','realtà aumenta','ar vr','vr ar','realta aumentata virtuale',
                                       'realtà virtuale ed aumentata','realtà aumentata o virtuale',
                                       'realtà aumentata virtuale','realtà aumentata e virtuale','realtà virtuale aumentata','augmented reality','augmentedreality','realta aumentata'],
                  'realtà mista':['mixed reality'],
                  'smart glasses':['occhiali intelligenti','smart glass','smart glassess'],
                  'gemello digitale':['digital twin'],
                  'avatar virtuale':['avatar digitali','avatar digitale','avatar virtuali'],
                  'metaverso':['metarverso', 'metaverses','metaverse'],
                  'nlp' : ['elaborazione del linguaggio naturale'],
                  'mondo virtuale':['ambiente virtuale'],
                  'realtà estesa':['xr'], 
                  'neurotecnologia':['neurotecnologie','neurotecnologico'], 
    'occhio artificiale':['occhi artificiali'],
        'ingegnerie neuromorfica':['calcolo neuromorfico']  },

           'nl' : {'virtual reality':['vr','virtual realityde','virtual reality-'],
                   'augmented reality':['augmented realiy'],
                   'virtuele avatar':['virtuele avatarser','virtuele avatarser','virtuele avatars'],
                   'virtuele wereld':['virtuele’ wereld'],
                   'virtuele omgeving':['virtuele omgevingen'],
                  
                    'verantwoordelijkheid':['zelfverantwoordelijkheid']},

           
           'ro':{
               'cyborg':['cyborgi'],'uter artificial':['uterul artificial'],
               'împădurire': ['împădurire','împădurirea','împăduriri',
                                'împădurim', 'împăduririle', 'îmădurire','2.împăduriri'],
                 'însămânţare a norilor' :['însămânţarea norilor',
                                           'însămânțarea norilor', 'însămânțare a norilor'],
                 'reîmpădurire':['reîmpădurirea','reîmpăduriri'],
                 'hidrogenul verde': ['hidrogen verde'],
                 'captare a carbonului' : ['captarea carbonului'],
                 'fotosinteza artificială':['fotosinteză artificială'],
                 'realitatea virtuală':['vr','realitate virtuală','realitate virtuala','realitate virtualăo'],
                 'realitate augmentată':['realitate augumentată','realitate augmentata','realitatea augmentată',
                                         'realitate augmentata','ar(realitate augmentată','realitatea augumentată'],
                 'ochelari inteligenți':['ochelarii inteligenți','ochelari inteligenţi','ochelarii inteligenţi',
                                         'ochelari smart','ochelarilor inteligenți','ochelari inteligent',
                                         'ochelari ar','ochelari inteligenti'],
                 'univers virtual':['universul virtual'],
                 'procesarea limbajului natural':['prelucrare a limbajului','nlp','prelucrarea limbajului natural'],
                 'metaverse':['metaversă','metaversu','metavers'],
                 'avatar virtual':['actor virtual','avatare','avatar'],
                 'lume virtuală':['lume virtuala','lumea virtuală','lumea virtuală','lume virtuala'],
                 'mediu virtual':['mediul virtual','univers virtual']},

           'rs':{'pošumljavanje':['pošumljavanja'],
                 'realitatea virtuală':['virtuelna realnost','virtual reality','vr','virtuelna stvarnost'],
                 'proširena realnost':['proširena stvarnost','ar'],
                 'procesarea limbajului natural':['neurolingvističko programiranje','obrada prirodnog jezika','nlp'],
                 'pametne naočare':['smart glasses'], 'moždani implant':['implant u mozgu'], 
               'moždani čip':['cxip u mozg']},

           'se':{'koldioxidavskiljning':['avskiljning av koldioxid'],

                 'virtual reality':['vr','virtuell verklighet'],
                 'metaverse':['metaversum'], 'augmented reality':['förstärkt verklighet']},

           'us': {'carbon capture': ['can capture','carbon   capture','carbon air capture',
                                     'carbon capturing','carbon be capture','carbon capt',
                                     'carboncapture','carbon recapture'], 
                  'green hydrogen':['green   hydrogen'],
                  'carbon storage': ['carbon store','carbon   store'],'cloud seeding':['cloud seed'],
                  'negative impact':['human might','negative impact'], 'ethics' : ['ethnic','ethic'],
                  'carbon sequestration':['carbon sequestering','carbon sequester in','co2 sequestration'],
                  'carbon usage':['carbon use','carbone usage'],
                  'sun shield':['sunshield'],
                  'solar radiation management':['radiation management'],

                  'virtual reality':['vr','virtual realities'],'digital avatar':['virtual avatar'],
                  'digital twin':['digital twins'],

                  'nlp':['natural language processing','natural language process'],
                  'virtual world':['virtual 3d world'],
                  'extended reality':['xr','extend reality'],
                  'smart glasses':['smart ar glass','smartglasse','smart sunglass','smart eyeglass','smart glass','smart glass'],
                  'metaverse':['metaverses','meataverse','metatverse'],
                  'mixed reality':['mix reality'],
                  'virtual environment':['virtual environments','virtual work environment'], 'augmented reality':['augment reality'],
                  
  'brain computer interface':['brain machine interface','brain to computer interface','brain computer interfaces','brain computer interfacing'],
    'neuroprosthetics':['neural prosthesis'],
        'cyborg':['cyborgs'],'neuroprosthetics':['neuroprosthetic','neural prosthesis'],
            'brain implant':['brain implants'],
                'neuromorphic engineering':['neuromorphic computing','neurmorphic computing'],
    'neurotechnology':['neurotech'],'neural control':['neurally control'],
                  'cognitive enhancement':['brain enhancement'],
            'brain chip':['brain chips']
                  
                 },


           'de' : {'neurotechnologie':['neurotech'],
                   'gehirnchip':['hirnchip','gehirnchips'], 
                   'gehirncomputerschnittstellen':['braincomputer interface','gehirncomputerschnittstelle',
                                                   'hirncomputerschnittstelle',
                                                   'hirncomputerschnittstelle',
                                                  'hirncomputerschnittstellen'],

                   "grüne wasserstoff":["grüner wasserstoff","grünen wasserstoff",
                                        "grünem wasserstoff","grünen wasserstoffs"],
                   "aufforstung" : ["aufforstungen",'aufforstungs'],
                   "metaversum" : ["metaversums", "metaverse",'virtueller welt'],
                   "virtuelle realität" : ["virtual reality","virtualreality",'vr',
                                           "virtueller realität","virtuellen realität",'virtuelle realitäten','virtuellen realitäten'],
                   "virtuelle welt" : ['virtuellen welt','virtuelle welten'],
                   "erweiterte realität" : ['augmented reality','augmented realitiy','erweitere realität',
                                            'erweiterte reality','erweiterten realitäten','augemted reality',
                                            'augumented reality',
                                            'augmentedreality','erweiterten realität','erweiterter realität'],
                   "virtuelle umgebung" : ['virtueller umgebung','virtuellen umgebungen','virtuellen umgebung','virtuelle umgebungen'],
                   'digitaler zwilling':['digitale zwillinge','digitalen zwillings','digitalen zwillinge',
                                         'digitalen zwilling',
                                        ],
                   'smart glasses' :   ['intelligente brillen','intelligente brille'],
                   'extended reality':['xr'],

                   'digitaler avatar':['digitale avatare','digitale avatar','digitalen avatar'],
                   'cyborg':['cyborgs'],
                       'würde':['würden','würnde'],
                   'rechts':['rechte','recht'],
                   'gesellschaft':['gesellschafter','gesellschaften'],
                   'datenschutz':['datenschutzes'],
                   'sicherheit' : ['sicherheits'],
                   'cloud seeding' : ['cloudseeding','wolkenbildung'],
                   'geoengineering':['geoengineerings'],
                   'klimatechnik' :['klimatechniker','climate engineering','climateengineerings'],

                   'virtual reality':['vr'],
                   'nlp':['natural language processing'],
                   'virtual world':['virtual 3d world'],
                   'extended reality':['xr','extend reality'],
                   'smart glasses':['smart ar glass','smartglasse','smart sunglass','smart eyeglass'],
                   'metaverse':['metaverses'],
                   'mixed reality':['mix reality'],
                   'virtual environment':['virtual environments']}
          }

In [9]:
def plot_ner_bars(df, remove, kw_col, country, country_code, topic, path, save = True):
    
    kw_list = get_one_list_kw_matches(df, kw_col, topic)
    
    counts = Counter(kw_list).most_common()
    
    xval = [w[0] for w in counts if w[0] not in remove ][0:20]
    yval = [w[1] for w in counts if w[0] not in remove][0:20]
    #xval = [w[0] for w in counts if any(c.isalpha() for c in w[0])][0:45]
    #yval = [w[1] for w in counts if any(c.isalpha() for c in w[0])][0:45]
    
    
    fig = go.Figure(data=[go.Bar(x=yval, y=xval, orientation='h')])
    fig.update_layout(
            title={
            'text': 'Named Entities counts - ' + kw_col + ' - ' + topic + ' ('+ country + ')',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            yaxis_title="Named Entities",
            xaxis_title="N mentions", 
             yaxis=dict(tickmode ='linear'))
    
    fig.update_yaxes(type='category')
    if save:
        IMG_NAME = kw_col + '_bar_ethics_' + topic + '.png'
        PATH_IMG = path + '/plots_raw/' + IMG_NAME
        fig.write_image(PATH_IMG)
    fig.show()
    
    return (counts, xval, yval)


def get_one_list_kw_matches(df, kw_col, topic,aliases_dict={}):
    
    kw_all = []
    for t in topics:
        for r in df[df['topic']==topic][kw_col]:
            if isinstance(r,list):
                for w in r:
                    w_add = str(w).lower().strip()
                    if len(w_add)>0:
                        kw_all.append(w_add)
            if isinstance(r,str):
                k = r[1:-1].split(',')
                for w in k:
                    w_add = w.lower().strip()
                    try:
                        w_add = eval(w_add)
                    except:
                        w_add = w_add
                    if len(w_add)>0:
                        kw_all.append(w_add)
    
    if len(kw_all)>0:
        kw_all = resolve_aliases(kw_all, aliases_dict)
    
    return kw_all

def convert_cell_to_list(cell):
    
    k = cell
    
    if isinstance(k, list)==False:
        try:
            k = eval(cell)
        except:
            k = cell[1:-1].split(',')
    
    return k
    

def get_list_kw_matches(df, kw_col, topic, aliases_dict):
    
    kw_all = []
    for i, r in df[df.topic==topic].iterrows():
        k = convert_cell_to_list(r[kw_col])
        k = [str(w).lower().strip() for w in k]
        k = resolve_aliases(k, aliases_dict)
        k = [" ".join(a.split()) for a in k]
        k = list(set(k))
        for w in k:
            kw_all.append(w)
        
    return kw_all

def plot_kw_matches_counts(df, kw_col, country, country_code, topic, path, nw=14,
                           orient='v',aliases_dict={},save=True, ethics =False):
    
    #kw_list = get_one_list_kw_matches(df, kw_col, topic,aliases_dict)
    #kw_list = lemmatize_list(kw_list)
    
    kw_list = get_list_kw_matches(df, kw_col, topic,aliases_dict)
    
    counts = Counter(kw_list).most_common()
    
    xval = [c[0].title() for c in counts][0:nw]
    yval = [c[1] for c in counts][0:nw]
    
    
    fig = go.Figure(data=[go.Bar(x=xval, y=yval, orientation=orient)])
    fig.update_layout(
            title={
            'text': 'Found keywords counts - ' + kw_col.split('_')[0] + ' - ' + topic + ' ('+ country + ')',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Keyword",
            yaxis_title="N matches" ,
    yaxis={'tickformat': ',d'})
    
    fig.update_xaxes(type='category')
    
    if save:
        IMG_NAME = kw_col + '_' + topic
        if ethics:
            IMG_NAME = IMG_NAME + '_ethics'
        PATH_IMG = path + '/plots_raw/' + IMG_NAME + '.png'
        fig.write_image(PATH_IMG)
    
    fig.show()
    
    return xval, yval, counts

def resolve_aliases(words_list, aliases_dict):
    
    for i in range(len(words_list)):
        for k,v in aliases_dict.items():
            if words_list[i] in v:
                words_list[i] = k
            if words_list[i] == 'grün    wasserstoff':
                words_list[i] = 'grün wasserstoff'
    
    return words_list

def get_fuzzy_matches(text, keywords, lang, ratio_thrs=90):
    
    nlp = spacy.blank(lang)
    doc = nlp(text)
    matcher = FuzzyMatcher(nlp.vocab)
    for kw in keywords:
        matcher.add("NOUN", [nlp(kw)])
    matches = matcher(doc)
    
    matches_list = []
    for match_id, start, end, ratio in matches:
        if ratio>ratio_thrs:
            matches_list.append(doc[start:end])
        
    #print('dirty matches')
    #print(matches_list)
    matches_list_clean = lemmatize_words_list(matches_list)
    
    return matches_list_clean

def get_matches_counts(str_list):
    
    n_words = len(str_list)
    unique_words = list(set(str_list))
    
    return n_words, len(unique_words), unique_words

def lemmatize_words_list(str_list):
    
    articles = ['lo','la','il','uno','una','uni','un']
    punct_pattern = re.compile(r"[^\w\s]")
    all_kw = []
    for kw in str_list:
        string = ''
        doc = nlp(str(kw))
        for w in doc:
            if w.lemma_ not in articles:
                string = string + str(w).lower() + ' '
        clean_string = punct_pattern.sub(r" ", string)
        clean_string = clean_string.replace('  ',' ')
        clean_string = clean_string.strip()
        all_kw.append(clean_string)
    
    return all_kw

def create_outlets_theme_df(df,outlets_dict):
    
    df_outlets = df.groupby(['topic','domain']).size().unstack('topic', fill_value=0).reset_index()
    df_outlets['domain_type'] = None
    df_outlets['domain_theme'] = None
    for o in outlets_dict.keys():
        df_outlets['domain_type'].loc[df_outlets['domain']==o] = outlets_dict[o]['type']
        df_outlets['domain_theme'].loc[df_outlets['domain']==o] = outlets_dict[o]['topic']
    df_outlets = df_outlets[(df_outlets['domain_theme'].notnull())]
    df_outlets['domain_theme_binary'] = df_outlets['domain_theme']
    df_outlets.loc[df_outlets['domain_theme']!='TECH',"domain_theme_binary"] = "NON_TECH"
    
    return df_outlets

def plot_outlets_theme_count(df_outlets, country, topic, path):
    
    xval = []
    yval = []
    df_tmp = df_outlets[['domain',topic,'domain_theme','domain_theme_binary']]
    for r in df_tmp.groupby("domain_theme").sum().iterrows():
        yvali = r[1][0]
        if yvali>0:
            yval.append(yvali)
            xval.append(r[0])
    
    fig = go.Figure(data=[go.Bar(x=xval, y=yval,text=yval)])
    fig.update_layout(
            title={
            'text': 'News outlets theme  - ' + topic + ' ('+ country + ')',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title="Outlet theme",
            yaxis_title="N news stories" )
    
    fig.update_xaxes(type='category')
    IMG_NAME = 'outlets_theme_' + topic + '.png'
    PATH_IMG = path + '/plots_raw/' + IMG_NAME
    fig.write_image(PATH_IMG)
    fig.show()
    
def lemmatize_string(string):
    
    doc = nlp(string)
    result = ' '.join([x.lemma_ for x in doc])
    
    return result

def lemmatize_list(str_list):
    
    #convert to string
    text = ','.join([k for k in str_list])
    # lemmatize
    text_lemm = lemmatize_string(text)
    # convert back to list
    str_list_lemm = [k.strip() for k in text_lemm.split(',')]
    
    return str_list_lemm

def get_fuzzy_matches_lemm(text, keywords, lang, ratio_thrs = 90):
    
    nlp = spacy.blank(lang)
    
    #remove punct
    punct_pattern = re.compile(r"[^\w\s]")
    text = punct_pattern.sub(r" ", text)
    
    # lemmatise text
    text  = lemmatize_string(text)
    # do this before as list is fixed
    doc = nlp(text)
    matcher = FuzzyMatcher(nlp.vocab)
    for kw in keywords:
        matcher.add("NOUN", [nlp(kw)])
    matches = matcher(doc)
    
    matches_list = []
    for match_id, start, end, ratio in matches:
        if ratio>ratio_thrs:
            matches_list.append(doc[start:end])

    return matches_list

def plot_bigrams(counts, remove, country, country_code, topic, path, save = True, name_add = ''):
    
    
    xval = [w[0] for w in counts if w[0] not in remove ][0:45]
    yval = [w[1] for w in counts if w[0] not in remove][0:45]
    
    
    fig = go.Figure(data=[go.Bar(x=yval, y=xval, orientation='h')])
    fig.update_layout(
            title={
            'text': 'Bigrams counts - ' + topic + ' ('+ country + ')',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            yaxis_title="Bigrams",
            xaxis_title="N mentions" )
    
    fig.update_yaxes(type='category')
    if save:
        IMG_NAME = 'bar_bigrams_' + topic + name_add + '.png'
        PATH_IMG = path + '/plots_raw/' + IMG_NAME
        fig.write_image(PATH_IMG)
    fig.show()
    
    return (xval, yval)

def remove_likely_tech_reviews(df, input_keywords):
    '''
    Given a dataframe, removes the rows that are likely to be tech products reviews.
    More precisely, it removes rows that contained custom tech-words in the title but 
    do not contain the keywords we are interested in (input_keywords) AND do not contain
    in title or in body, any ethical keyword
    '''
    ids_remove = get_likely_tech_reviews_ids(df, input_keywords)
    df_new = df.drop(df.index[ids_remove]).reset_index(drop=True)
    
    return df_new
    

def get_likely_tech_reviews_ids(df, input_keywords):
    
    tech_words = ['iOS', 'Android', 'iPad', 'iPhone', 'Play Store', 'Pixel','Xiaomi', 'Oppo','Samsung']
    too_tech_news_id = []
    for i, r in df.iterrows():
        if len(set(tech_words).intersection(set(r['title'].split())))>0:
            if len([w for w in input_keywords if w.lower() in r['title'].lower()])==0:
                if (len(r['ethics_matches']) <=1):
                    too_tech_news_id.append(i)
    
    return too_tech_news_id

def get_keywords_country_topic_list(keywords_dict, topic, language_key):

    kw_list = [s.strip().strip('"') for s in keywords_dict[topic][language_key] if str(s) != 'nan']
    
    if language_key not in ['GERMAN_DE','GERMAN_AT']:
        kw_list = [s.lower() for s in kw_list]
        
    
    return kw_list